Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# Set Cover problem

See: https://en.wikipedia.org/wiki/Set_cover_problem

In [2]:
from random import random, seed
from itertools import product
import numpy as np

from icecream import ic

## Reproducible Initialization

If you want to get reproducible results, use `rng` (and restart the kernel); for non-reproducible ones, use `np.random`.

In [3]:
UNIVERSE_SIZE = 10000
NUM_SETS = 1000
DENSITY = 0.2

rng = np.random.Generator(np.random.PCG64([UNIVERSE_SIZE, NUM_SETS, int(10_000 * DENSITY)]))

In [4]:
# DON'T EDIT THESE LINES!

SETS = np.random.random((NUM_SETS, UNIVERSE_SIZE)) < DENSITY
for s in range(UNIVERSE_SIZE):
    if not np.any(SETS[:, s]):
        SETS[np.random.randint(NUM_SETS), s] = True
COSTS = np.power(SETS.sum(axis=1), 1.1)

## Helper Functions

In [5]:
def valid(solution):
    """Checks wether solution is valid (ie. covers all universe)"""
    return np.all(np.logical_or.reduce(SETS[solution]))


def cost(solution):
    """Returns the cost of a solution (to be minimized)"""
    return COSTS[solution].sum()

## Have Fun!

In [6]:
# A dumb solution of "all" sets
solution = np.full(NUM_SETS, True)
valid(solution), cost(solution)

(True, 4275648.283734298)

In [7]:
# A random solution with random 50% of the sets
solution = rng.random(NUM_SETS) < .5
valid(solution), cost(solution)

(True, 2141303.2708097063)

### Solution

In [8]:
# Tweak and restart

# Best solution - initializes with all sets
best = np.full(NUM_SETS, True)

# How many times it should restart
max_restarts = 100

for i in range(max_restarts):

    # Initialize the solution (start with no sets selected)
    solution = np.full(NUM_SETS, False)

    # Keep adding sets until the solution is valid
    while not valid(solution):
        # Randomly pick a set that is not already in the solution
        index = np.random.randint(0, NUM_SETS)
        if not solution[index]:  # Only add if not already in the solution
            solution[index] = True
        
           

    # Return the valid solution and its cost
    #print(valid(solution), cost(solution))

    if cost(solution) < cost (best):
        best = solution.copy()


print(valid(best), cost(best))

True 153406.37677072093


In [9]:
# Stronger tweak and restart

# Best solution - initializes with all sets
best = np.full(NUM_SETS, True)

# How many times it should restart
max_restarts = 100

for i in range(max_restarts):

    # Initialize the solution (start with no sets selected)
    solution = np.full(NUM_SETS, False)

    # Keep adding sets until the solution is valid
    while not valid(solution):
        index = None
        while index is None or np.random.random() < 0.4:
            index = np.random.randint(0, NUM_SETS)
        
            if not solution[index]:  # Only add if not already in the solution
                solution[index] = True

    # Return the valid solution and its cost
    #print(valid(solution), cost(solution))

    if cost(solution) < cost (best):
        best = solution.copy()


print(valid(best), cost(best))

True 153325.5447269273


In [10]:
# Greedy algorithm

# Initialize the solution (start with no sets selected)
solution = np.full(NUM_SETS, False)

# Array with all elements uncovered (initializes with all elements)
uncovered_elements = np.where(np.any(SETS, axis=0))[0]


while not valid(solution):

    # Cost of the best set (initializes with the cost of all sets)
    best_set_cost = cost(np.full(NUM_SETS, True))

    # Maximum number of uncovered elements in a set
    max_uncovered = 0

    # Set with more uncovered elements and lowest cost
    best_set_index = None

    for set_index in range(len(SETS)):

        # Checks if set has already been used
        if not solution[set_index]:

            # Number of elements still uncovered that a set covers
            uncovered_elements_in_set = len(np.intersect1d(uncovered_elements, np.where(SETS[set_index])[0]))
            
            # Set cost 
            set_cost = COSTS[set_index]

            # Update the best set based on uncovered elements and cost
            if uncovered_elements_in_set > max_uncovered:
                max_uncovered = uncovered_elements_in_set
                best_set_index = set_index
                best_set_cost = set_cost
            elif uncovered_elements_in_set == max_uncovered and set_cost < best_set_cost:
                best_set_index = set_index
                best_set_cost = set_cost
    
    # Adds set to solution
    solution[best_set_index] = True

    # Updates list of uncovered elements
    uncovered_elements = np.setdiff1d(uncovered_elements, np.where(SETS[best_set_index])[0])
    

print(valid(solution), cost(solution))        
        

True 104233.53574710633


In [16]:
# Randomly removes one set at a time

# Initialize the solution (start with all sets selected)
solution = np.full(NUM_SETS, True)  

# Perform the random removal process multiple times
num_removals = 100  

for _ in range(num_removals):

    new_solution = solution.copy()

    # Randomly pick a subset to remove
    set_to_remove = np.random.randint(0, NUM_SETS)
    new_solution[set_to_remove] = False

    # Check if the new solution is still valid
    if valid(new_solution):
        solution = new_solution


print(valid(solution), cost(solution))


True 3864858.7690510782


In [23]:
# Randomly removes sets

# Initialize the solution (start with all sets selected)
solution = np.full(NUM_SETS, True)  

# Perform the random removal process multiple times
num_removals = 100  

for _ in range(num_removals):

    new_solution = solution.copy()

    # Randomly remove sets with a 90% chance for additional removals
    index = None
    while index is None or np.random.random() < 0.9:
        # Pick a random set to remove
        index = np.random.randint(0, NUM_SETS)

        # Only remove if it's currently in the solution 
        if new_solution[index]:
            new_solution[index] = False

    # Check if the new solution is still valid
    if valid(new_solution):
        solution = new_solution

print(valid(solution), cost(solution))


True 1444421.3221872877
